## Part 1 - Data Preparation and Exploration 

In [ ]:
%%capture
# Due to the configuration of the base Jupter image, the following imports are required for the regressions in the assignment to report the correct metrics

import sys 
!{sys.executable} -m pip uninstall statsmodels --yes 
!{sys.executable} -m pip uninstall numpy --yes
!{sys.executable} -m pip uninstall pandas --yes 
!{sys.executable} -m pip uninstall patsy --yes 
!{sys.executable} -m pip install numpy==1.17
!{sys.executable} -m pip install pandas==1.0
!{sys.executable} -m pip install patsy==0.5.2
!{sys.executable} -m pip install statsmodels==0.11.1

In [ ]:
#Import Libraries

import pandas as pd
import datetime as dt
import scipy.stats as sp
import numpy as np
pd.reset_option("all")
import statsmodels.formula.api as sm 

In [ ]:
# Import Shotlog_14_15 and Player_Stats Datasets

Shotlog_1415=pd.read_csv("Assignment Data/Week 6/Shotlog_14_15.csv")
Player_Stats=pd.read_csv("Assignment Data/Week 6/Player_Stats_14_15.csv")
display(Shotlog_1415)
Shotlog_1415['game_id'].nunique()

In [ ]:
Shotlog_1415['current_shot_outcome']
Shotlog_1415['current_shot_outcome']=np.where(Shotlog_1415['current_shot_outcome']=='made', 1, 0)
Shotlog_1415.info()

In [ ]:
import datetime as dt
Shotlog_1415['date']=pd.to_datetime(Shotlog_1415['date'])
Shotlog_1415['date'].head(10)

In [ ]:
Shotlog_1415["game_clock"].head()

In [ ]:
Shotlog_1415["lag_shot_hit"]=Shotlog_1415.sort_values(by=['shot_number'], ascending=[True]).groupby(['shoot_player', 'date'])['current_shot_outcome'].shift(1)
Shotlog_1415['lag_shot_hit'].head(20)

In [ ]:
Shotlog_1415.sort_values(by=['shoot_player', 'game_id', 'shot_number'], ascending=[True, True, True])

In [ ]:
Shotlog_1415["error"]=Shotlog_1415['current_shot_outcome']-Shotlog_1415['average_hit']
Shotlog_1415["lagerror"]=Shotlog_1415['lag_shot_hit']-Shotlog_1415['average_hit']
Shotlog_1415.head(10)


In [ ]:

Shotlog_1415['error'].describe()

In [ ]:
Shotlog_1415['lagerror'].describe()

## Part 2 - Conditional Probability and Autocorrelation

In [ ]:
Shotlog_1415=Shotlog_1415[pd.notnull(Shotlog_1415["lag_shot_hit"])]
Shotlog_1415['conse_shot']=np.where((Shotlog_1415['current_shot_outcome']==1)&(Shotlog_1415['lag_shot_hit']==1),1,0)
Shotlog_1415['conse_shot']

In [ ]:
Player_Prob=Shotlog_1415.groupby(['shoot_player'])['conse_shot','lag_shot_hit'].mean()
Player_Prob=Player_Prob.reset_index()
Player_Prob.rename(columns={'lag_shot_hit':'average_lag_hit', 'conse_shot':'conse_shot_hit'}, inplace=True)
Player_Prob.head()

In [ ]:
Player_Prob['conditional_prob']=Player_Prob['conse_shot_hit']/Player_Prob['average_lag_hit']


In [ ]:
Player_Stats=pd.merge(Player_Prob, Player_Stats, on=['shoot_player'])
Player_Stats=Player_Stats[pd.notnull(Player_Stats["conditional_prob"])]
Player_Stats.head(10)

In [ ]:
Player_Stats['diff_prob']=Player_Stats['conditional_prob']-Player_Stats['average_hit']
Player_Stats['diff_prob'].describe()

In [ ]:
Player_Stats['average_hit'].describe()

In [ ]:
Player_Stats['conditional_prob'].describe()

In [ ]:
Player_Stats['conse_shot_hit'].describe()

In [ ]:
import scipy.stats as sp
sp.stats.ttest_ind(Player_Stats['conditional_prob'], Player_Stats['average_hit'])

In [ ]:
Shotlog_1415['current_shot_hit'].corr(Shotlog_1415['lag_shot_hit'])

In [ ]:
Autocorr_Hit=Shotlog_1415.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().unstack().iloc[:,1].reset_index()
Autocorr_Hit.columns=Autocorr_Hit.columns.get_level_values(0)
Autocorr_Hit.rename(columns={'current_shot_hit':'autocorr'}, inplace=True)
Autocorr_Hit.sort_values(by=['autocorr'], ascending=[False])


## Part 3 - Regression Analyses

In [ ]:
Reg1 = sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+points_earned+C(quarter)+home_away+C(shoot_player)+C(closest_defender)+closest_def_dist', data= Shotlog_1415).fit()
print(Reg1.summary())


In [ ]:
Reg2 = sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+points_earned+C(quarter)+home_away+C(shoot_player)+C(closest_defender)+closest_def_dist', weights=1/Shotlog_1415['shot_per_game'] ,data= Shotlog_1415).fit()
print(Reg2.summary())

In [ ]:
def reg_player(player):
    Shotlog_player=Shotlog_1415[Shotlog_1415.shoot_player==player]
    reg_player=sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+points_earned+C(quarter)+home_away+closest_def_dist', data= Shotlog_player).fit()
    print(reg_player.summary())
    return; 
def reg_wls_player(player):
    Shotlog_player=Shotlog_1415[Shotlog.shoot_player==player]
    reg_wls_player=sm.wls(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+points_earned+C(quarter)+home_away+closest_def_dist',weights=1/Shotlog_player['shot_per_game'] , data= Shotlog_player).fit()
    print(reg_wls_player.summary())
    return; 

In [ ]:
Reg3_player('Russell Westrbrook')
Reg4_wls_player('Kobe Bryant')